In [1]:
import os
import sys
import time
import datetime
import socket
import yaml
import shutil
from sklearn.externals import joblib 

feaName = 'wtio'
sys.path.append('./wavelet/wtio'); import wtio
extractor = wtio

In [2]:
#feaPath = '../xprmt/feature/extract-wtio-eomf-mlbox-20180924-124821/eomf'
feaPath = '../xprmt/feature/extract-wtio-acacia-crassicarpa-mlbox-20180926-001202/acacia-crassicarpa'

xList = []
yList = []
yLi = os.listdir(feaPath)
sumdat = 0
for i in yLi:
    file = os.listdir(feaPath+ "/" + i)
    cnt = 0
    for j in file:
        if j.endswith('.pkl'):
            #print(j)
            x = joblib.load(feaPath + "/" + i + "/" +j)
            xList.append(x)
            yList.append(i)
            cnt += 1
    print(str(i) + " " + str(cnt))
    sumdat += cnt
print("semuanya "+ str(sumdat))
    

bercak-bulat 328
sehat 247
bercak-garis 328
semuanya 903


In [3]:
import numpy as np
xList = np.array(xList)
yList = np.array(yList)

In [4]:
print(len(xList))
print(len(yList))

903
903


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xList, yList, random_state = 42, test_size = 0.2)
print(len(X_train))
print(len(X_test))

722
181


In [6]:
X_train.shape

(722, 8184)

In [7]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
prediksi = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, prediksi))

0.8784530386740331


In [9]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf, xList, yList, cv=10))

[0.85714286 0.78021978 0.68131868 0.85714286 0.79120879 0.81318681
 0.83516484 0.85555556 0.82954545 0.84090909]


In [10]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
    ########################## perform 10 fold validation ######################
    kf = KFold(n_splits=10)
    scorelist = []
    for train_index, test_index in kf.split(xList):
        clf.fit(xList[train_index], yList[train_index])
        p = clf.predict(xList[test_index])
        accval = accuracy_score(yList[test_index], p)
        scorelist.append(accval)
    
    print("MeanCVScore: {}".format(sum(scorelist)/len(scorelist)))
    print("10FoldCVScore: {}".format(scorelist))
    #############################################################################
    
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 75.6906%
Log Loss: 2.6434312080283235
MeanCVScore: 0.6344444444444445
10FoldCVScore: [0.6043956043956044, 0.7582417582417582, 0.6373626373626373, 0.7888888888888889, 0.7777777777777778, 0.7666666666666667, 0.6222222222222222, 0.37777777777777777, 0.5666666666666667, 0.4444444444444444]
SVC
****Results****
Accuracy: 55.8011%
Log Loss: 0.7970162180377894


KeyboardInterrupt: 